### **5.11 - [Practica] Pools and priority_weights: Limiting parallelism – prioritizing tasks**

En este vídeo vamos a descubrir los pools y por qué son útiles.  Un pool es una gran manera de limitar el número de instancias concurrentes de un tipo específico de tarea para evitar saturar el origen o el destino.  Por ejemplo, digamos que tienes tres nodos worker, donde en total se pueden ejecutar tres tareas en paralelo.  

<center><img src="https://i.postimg.cc/50v306gt/a908.png"></center>

Ahora, en tu DAG, tienes tres tareas para extraer datos de una API REST en paralelo, pero esta API sólo puede solicitarse una vez cada vez.  La pregunta es, ¿cómo puede limitar el número de solicitudes a 1 sin tener que modificar los parámetros de su instancia de Airflow?  Pues bien, ahí es donde los pools vienen a salvarle. Utilizando pools, puede limitar el número de tareas que solicitan la API definiendo un número de slots, que en nuestro caso es 1, y asignar las tres tareas a este pool.  Al hacerlo, las tres tareas se ejecutarán secuencialmente una tras otra.  Veamos un ejemplo de esto.  En primer lugar, comprueba que estás en la carpeta airflow-materials/airflow-section-5 e inicia los contenedores docker ejecutando el siguiente comando "docker-compose -f docker-compose-CeleryExecutor.yml up -d --scale worker=3".  Este comando iniciará Airflow con tres nodos worker. De acuerdo. 

<center><img src="https://i.postimg.cc/28CwkjZz/a909.png"></center>

Ahora echemos un vistazo al DAG que vamos a utilizar. Desde tu editor de código, abre el DAG pool_dag.py en la carpeta mnt/airflow/dags. Este DAG es bastante emocionante ya que he utilizado algunos conceptos que puede que no conozcas. No te preocupes, los veremos en las próximas secciones. Así pues, tenemos tres tareas que son tres SimpleHttpOperators que solicitarán una forex API para obtener los tipos de cambio de una divisa determinada (rates of a given currency). La tarea "get_forex_rate_EUR" obtiene los tipos del EUR, luego "get_forex_rate_USD" obtiene los tipos del USD, y tenemos la misma tarea para el JPY.  El resultado de cada petición (request) se almacenará utilizando XComs.  Eso es lo que se puede ver por el parámetro xcom_push que se muestra aquí.  Vamos a ver lo que son, pero por ahora sólo tener en cuenta que es una manera de compartir datos entre tareas en Airflow. Al final del DAG, tenemos una tarea llamada "show_data" que obtendrá las tasas almacenadas en la metadata database de Airflow y las mostrará en la salida.  Usted puede estar asustado por el comando bash que se muestra aquí, pero básicamente estoy haciendo un bucle a través de las diferentes tareas del DAG para obtener los datos que almacenan. No te preocupes, hay un video dedicado sobre templating y macros más adelante en el curso.  Por último, para poner en marcha el DAG, tenemos que crear una conexión llamada "forex_api" como se define aquí.  

<center><img src="https://i.postimg.cc/sx44Yqtd/a910.png"></center>
<center><img src="https://i.postimg.cc/Bb0C147B/a911.png"></center>

Muy bien vamos a hacer esto.  Abra su navegador web, y vaya a la interfaz de usuario de Airflow escribiendo localhost:8080.  Enter. Haga clic en "admin", "connections" y "create". En "Conn Id" escriba "forex_ api".  "Conn type" es igual a "HTTP" y el host se establece en "api.exchangeratesapi.io". Ok, haga clic en "Guardar". 

<center><img src="https://i.postimg.cc/2jQGNL3c/a912.png"></center>

Vuelve a la DAGs view. Activa el DAG pool_dag y actualiza la página.  Ahora espera a que termine el DAGRun.  Perfecto.  

<center><img src="https://i.postimg.cc/wBR5f7Sk/a913.png"></center>

Si pulsamos sobre él y vamos a la Gantt view, podemos ver que las tres tareas se han ejecutado en paralelo como esperábamos.  Ahora queremos limitar el número de tareas que llegan concurrentemente a la API a 1. 

<center><img src="https://i.postimg.cc/Z5RxNfH2/a914.png"></center>

Para ello vamos a "Admin", "Pools" y pinchamos en crear.  Aquí vamos a crear un pool llamado "forex_api_pool" con 1 slot, y en la descripción, vamos a decir "pool para limitar el número de peticiones a la API".  Ok, click en guardar.  

<center><img src="https://i.postimg.cc/TYQcp8Vn/a915.png"></center>

Entonces tenemos el nuevo pool "forex_api_pool" como se muestra aquí.  El "default_pool" es el pool por defecto usado por tus tareas limitando el número de tareas concurrentes a 128. Esto significa que, si estableces el número de slots de ese pool a 1, podrás ejecutar 1 tarea a la vez.  Ok ahora tenemos nuestro pool definido, tenemos que mover a las tareas dentro de él. 

<center><img src="https://i.postimg.cc/kgpFvxh4/a916.png"></center>

Vuelve a tu editor de código, para cada tarea get_rate, añade el parámetro "pool="forex_api_pool"".  No te olvides de las dos comillas dobles y hacemos esto para todas las tareas.  Ok, guarde el archivo.  

<center><img src="https://i.postimg.cc/j5VHGRTY/a917.png"></center>

Ahora vuelva a la interfaz de usuario de Airflow, haga clic en DAGs y dispara el dag pool_dag haciendo clic en este botón.  Ahora empieza a refrescar la página hasta que el DAGRun haya terminado.  Ok, ya está hecho. 

<center><img src="https://i.postimg.cc/vTh5g9VG/a918.png"></center>

Haga clic en él, "Gantt View" y como se puede ver, las tareas se han ejecutado secuencialmente, una tras otra sin cambiar los parámetros de Airflow. 

<center><img src="https://i.postimg.cc/L52z8vhy/a919.png"></center>

**`Así que los pools son útiles para limitar el número de tareas concurrentes en un ámbito concreto que tú definas`**. Ah, y puedes comprobar la salida dada por la tarea show_data haciendo clic en la tarea desde el tree view, y luego "view logs".  

<center><img src="https://i.postimg.cc/vBWryC8g/a920.png"></center>

Y si te desplazas hacia abajo, puedes ver los índices que obtuvimos.  

<center><img src="https://i.postimg.cc/fWfcr4g1/a921.png"></center>

Ok, antes de seguir adelante, me gustaría mostrar cómo se puede dar prioridad a las tareas en tu pool. En realidad, hay un parámetro que puede añadir a cualquier operador que es **`priority_weight`**.  Este parámetro le permite definir el orden en la cola y que tareas se ejecutan primero como slots abiertos en el pool. Por defecto, este priority_weight está establecido en 1.  Lo que podemos hacer, es cambiar este valor a cualquier número para establecer el orden en que las tareas van a ser ejecutadas dentro del pool.  Vaya a su editor de código, y para la tarea "get_forex_rate_EUR" definimos el "priority_weight" a 1. 

<center><img src="https://i.postimg.cc/qMBsfmPq/a922.png"></center>

Luego, para la tarea "get_forex_rate_USD" establecemos el parámetro a 2. Y finalmente, para la tarea "get_forex_rate_JPY", establecemos el parámetro al valor 3.  De este modo, obtendremos primero los tipos de cambio del JPY (rates of JPY), después los del USD y finalmente los del EUR. Guarde el archivo y vamos a ejecutar el DAG. Haga clic en "DAGs", a continuación, haga clic en el botón aquí, para activar el DAG. Actualiza la página hasta que el DAGRun haya terminado.  Ok, ahora haz clic en el DAG, 

<center><img src="https://i.postimg.cc/tJJ4kVcq/a904.png"></center>

luego en "Gantt view" y como puedes ver, el orden es JPY, USD y EUR como se esperaba. Hemos definido la tarea "get_forex_rate_JPY" como la más importante a ejecutar en primer lugar. 

<center><img src="https://i.postimg.cc/D0NyMwVq/a905.png"></center>

Puede comprobar que el orden ha cambiado simplemente eligiendo la ejecución anterior de la lista de aquí, haga clic en " GO ". 

<center><img src="https://i.postimg.cc/SNXKzC8V/a906.png"></center>

Y aquí, la tarea USD se ejecutó primero.  

<center><img src="https://i.postimg.cc/dQN1147L/a907.png"></center>

Muy bien, así que ha aprendido cómo puede limitar la concurrencia entre diferentes tareas sin cambiar los parámetros de Airflow con pools.  También ahora es capaz de dar prioridad a las tareas en un grupo determinado para darle aún más flexibilidad. Antes de pasar al siguiente vídeo, vaya a su terminal, y detenga los contenedores docker escribiendo "docker-compose - f docker-compose-CeleryExecutor.yml down".  OK.  Espero que hayas disfrutado de este video, vamos a tomar un breve descanso y nos vemos para un ejecutor muy emocionante.  